In [1]:
from __future__ import print_function
from misc import *



In [2]:
"""DVS128 closed loop oscillator routine by Alexander Rivkind, Eldad Assa, Michael Kreiserman and Ehud Ahissar
based on: 
*DVS example by: Yuhuang Hu (duguyue100@gmail.com)
*"Syclop" paradigm: Ahissar and Assa 2017
"""
from reservoir import ESN
import numpy as np
import time
import SYCLOP_env as syc
import matplotlib.pyplot as plt


import cv2
from misc import *
import maestro
import teachers
from pyaer.dvs128 import DVS128
%matplotlib notebook

In [3]:
import pickle

In [4]:
with open('games_s2.pkl','rb') as f:
    esn0 = pickle.load(f)
with open('games_s2.pkl','rb') as f:
    esn1 = pickle.load(f)

In [5]:
A_net_teacher = 5
servo = maestro.Controller()

In [6]:
def gohome():    
    servo.setSpeed(0,0)
    servo.setSpeed(1,0)
    servo.setTarget(1,7500)     #set speed of servo 1
    servo.setTarget(0,5000)  #set servo to move to center position
    servo.setSpeed(0,1)
    servo.setSpeed(1,1)
    time.sleep(1.0)

In [7]:
training = False
device = DVS128()


device.start_data_stream()
# load new config
# device.set_bias_from_json("./scripts/configs/dvs128_config.json")
# print (device.get_bias())

clip_value = 3
histrange = [(0, v) for v in (128, 128)]
t_prev = time.time()



dt_nom = 0.025
# dt_list = []
cnt = 0
empty_cnt = 0

gohome()
while True:
    t_this = time.time()
    try:
        (pol_events, num_pol_event,
         special_events, num_special_event) = \
            device.get_event("events_hist")
        if t_this - t_prev > dt_nom:
                if num_pol_event != 0:
                    img = pol_events[..., 1]-pol_events[..., 0]
                    observation0=np.sum(img**2,axis=0)
                    observation1=np.sum(img**2,axis=1)
                    esn0.step(uin = observation0)
                    esn1.step(uin = observation1)

                    servo.moveByVelocity(0,(int(np.round(A_net_teacher*esn0.z))))
                    servo.moveByVelocity(1,(int(np.round(A_net_teacher*esn1.z))))

                    img = np.clip(img, -clip_value, clip_value)
                    img = img+clip_value
                    print(cnt,'/',empty_cnt,esn1.z,'\r',end='')
                    cv2.imshow("image", img/float(clip_value*2))
                    t_prev=t_this
                    if cv2.waitKey(1) & 0xFF == ord('q'):
                        break
                    cnt += 1
        else:
            empty_cnt += 1

    except KeyboardInterrupt:
            device.shutdown()
            break


In [8]:
# import pickle

In [9]:
# with open('games_u1.pkl','wb') as f:
#     pickle.dump(esn,f)

In [10]:
np.sum(esn.wout*esn.wfb)

NameError: name 'esn' is not defined

In [ ]:
np.max(np.imag(np.linalg.eig(esn.W)[0]))

In [ ]:
np.flip([1,2,3,4,5])

In [ ]:
esn1.z